In [1]:
import pandas as pd

In [2]:
import geopandas as gpd # type: ignore

def carregar_geojson_em_dataframe(caminho_arquivo):
    try:
        # Carrega o GeoJSON usando o geopandas
        gdf = gpd.read_file(caminho_arquivo)
        return gdf
    except Exception as e:
        print(f"Erro ao abrir o arquivo GeoJSON: {e}")
        return None

acidentes_df = carregar_geojson_em_dataframe("../data/ANTT/BR-101-SC/acidentes_br101sc.geojson")
acidentes_df.head(2)

/home/guibatalhoti/Dados_ANTT/_env/lib/python3.12/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


,n_da_ocorrencia,km,trecho,sentido,tipo_de_acidente,automovel,bicicleta,caminhao,moto,onibus,...,mortos,concessionaria,data_hora,com_vitima,estado,ano,mes,latitude,longitude,geometry
0,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,0,viacosteira,2021-05-01 12:10:00,False,SC,2021,5,-28.425440,-48.857996,POINT (-48.85800 -28.42544)
1,1,316.0,BR-101/SC,Decrescente,Saída de pista,2,0,0,0,0,...,0,viacosteira,2021-05-01 12:10:00,False,SC,2021,5,-28.426395,-48.859912,POINT (-48.85991 -28.42639)


In [3]:
# conta a quantidade de valores vazios em latitude
acidentes_df['latitude'].isnull().sum()

539

In [4]:
acidentes_df.columns

Index(['n_da_ocorrencia', 'km', 'trecho', 'sentido', 'tipo_de_acidente',
       'automovel', 'bicicleta', 'caminhao', 'moto', 'onibus', 'outros',
       'tracao_animal', 'transporte_de_cargas_especiais', 'trator_maquinas',
       'utilitarios', 'ilesos', 'levemente_feridos', 'moderadamente_feridos',
       'gravemente_feridos', 'mortos', 'concessionaria', 'data_hora',
       'com_vitima', 'estado', 'ano', 'mes', 'latitude', 'longitude',
       'geometry'],
      dtype='object')

In [5]:
estacoes_proximas = carregar_geojson_em_dataframe("estacoes_inmet.geojson")
estacoes_proximas

/home/guibatalhoti/Dados_ANTT/_env/lib/python3.12/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


,regiao,estado,codigo_estacao,nome_estacao,lat,long,alt,data_fundacao,nome_arquivo,geometry
0,S,RS,MOSTARDAS,A878,-31.248333,-50.906389,3.82,2008-03-11,../data/INMET/2022/INMET_S_RS_A878_MOSTARDAS_0...,POINT (-50.90639 -31.24833)
1,CO,GO,ITUMBIARA,A035,-18.409722,-49.191944,491.17,2007-11-01,../data/INMET/2022/INMET_CO_GO_A035_ITUMBIARA_...,POINT (-49.19194 -18.40972)
2,S,RS,DOM PEDRITO,A881,-31.002500,-54.618056,150.00,2010-04-23,../data/INMET/2022/INMET_S_RS_A881_DOM PEDRITO...,POINT (-54.61806 -31.00250)
3,CO,MT,VILA BELA DA SANTISSIMA TRINDADE,A922,-15.062778,-59.873056,213.00,2006-12-01,../data/INMET/2022/INMET_CO_MT_A922_VILA BELA ...,POINT (-59.87306 -15.06278)
4,SE,MG,ALMENARA,A508,-16.166667,-40.687778,189.11,2002-12-15,../data/INMET/2022/INMET_SE_MG_A508_ALMENARA_0...,POINT (-40.68778 -16.16667)
...,...,...,...,...,...,...,...,...,...,...
2871,S,RS,CAMAQUA,A838,-30.807953,-51.834240,92.30,2006-12-12,../data/INMET/2021/INMET_S_RS_A838_CAMAQUA_01-...,POINT (-51.83424 -30.80795)
2872,NE,BA,BOM JESUS DA LAPA,A418,-13.251111,-43.405278,447.75,2007-05-18,../data/INMET/2021/INMET_NE_BA_A418_BOM JESUS ...,POINT (-43.40528 -13.25111)
2873,N,AM,S. G. DA CACHOEIRA,A134,-0.125207,-67.061246,79.67,2011-08-31,../data/INMET/2021/INMET_N_AM_A134_S. G. DA CA...,POINT (-67.06125 -0.12521)
2874,SE,MG,PASSOS,A516,-20.745237,-46.633916,781.70,2006-07-16,../data/INMET/2021/INMET_SE_MG_A516_PASSOS_01-...,POINT (-46.63392 -20.74524)


In [6]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

def find_nearest_weather_station(accidents_df, weather_stations_df, radius_km=50):
    # Copy the input DataFrames to avoid modifying the original DataFrames
    accidents_df = accidents_df.copy()
    weather_stations_df = weather_stations_df.copy()

    # Remover linhas com valores NaN em coordenadas
    accidents_df = accidents_df.dropna(subset=['latitude', 'longitude'])
    weather_stations_df = weather_stations_df.dropna(subset=['lat', 'long'])

    # Lista para armazenar resultados
    merged_data = []

    # Itera sobre cada linha do DataFrame de acidentes
    for _, accident_row in accidents_df.iterrows():

        accident_location = (accident_row['latitude'], accident_row['longitude'])

        #print(f"Procurando estação climática mais próxima para o acidente em {accident_location}")

        # Calcula a distância para cada estação climática
        weather_stations_df['distance'] = weather_stations_df.apply(
            lambda row: geodesic(accident_location, (row['lat'], row['long'])).kilometers, axis=1
        )

        # Filtra estações dentro do raio especificado
        nearby_stations = weather_stations_df[weather_stations_df['distance'] <= radius_km]

        if not nearby_stations.empty:
            # Encontra a estação mais próxima
            nearest_station = nearby_stations.loc[nearby_stations['distance'].idxmin()]
            combined_row = {**accident_row.to_dict(), **nearest_station.to_dict()}
        else:
            # Se nenhuma estação estiver próxima, preencha com NaN
            combined_row = {**accident_row.to_dict(), **{col: np.nan for col in weather_stations_df.columns}}

        merged_data.append(combined_row)

    # Retorna o DataFrame combinado
    return pd.DataFrame(merged_data)

# acidentes_sample = acidentes_df.sample(15)
# datas_climaticas = pd.read_csv("datas_climaticas.csv")

acidentes_com_estacoes = find_nearest_weather_station(acidentes_df, estacoes_proximas)
acidentes_com_estacoes

KeyboardInterrupt: 

In [ ]:
acidentes_com_estacoes.to_csv("./secundario/acidentes_com_estacoes.csv", index=False)

In [ ]:
import pandas as pd
import os
import re

def load_and_concat_unique_data_with_metadata(df, years=[2020, 2021, 2022, 2023]):
    """
    Lê arquivos CSV baseados em valores únicos na coluna 'nome_arquivo' de um DataFrame,
    processa para anos especificados e concatena sem duplicações, adicionando as informações
    das 8 primeiras linhas como novas colunas.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo a coluna 'nome_arquivo' com caminhos dos arquivos.
        years (list): Lista de anos de interesse para filtrar os dados (default: [2020, 2021, 2022, 2023]).

    Retorna:
        pd.DataFrame: Um DataFrame concatenado sem duplicações, com as colunas extras.
    """
    # Obter valores únicos na coluna 'nome_arquivo'
    unique_files = df['nome_arquivo'].dropna().unique()

    # Inicializa o DataFrame vazio para concatenar os resultados
    concatenated_df = pd.DataFrame()

    # Expressão regular para encontrar o ano no nome do arquivo
    year_pattern = r'\d{4}'  # padrão para encontrar ano de 4 dígitos

    # Conjunto para armazenar arquivos processados e evitar duplicações
    processed_files = set()

    for base_path in unique_files:
        # Encontra o ano presente no nome do arquivo
        match = re.search(year_pattern, base_path)
        if match:
            current_year = match.group()  # Ano encontrado no caminho do arquivo
            
            for year in years:
                # Substitui o ano encontrado no caminho do arquivo pelo ano da iteração
                file_path = base_path.replace(current_year, str(year))

                if file_path not in processed_files and os.path.exists(file_path):
                    print(f"Lendo arquivo: {file_path}")
                    processed_files.add(file_path)  # Marca o arquivo como processado
                    
                    try:
                        # Lê as primeiras 8 linhas do arquivo para extrair as variáveis adicionais
                        with open(file_path, 'r', encoding='latin1') as file:
                            first_8_lines = [next(file) for _ in range(8)]

                        # Extraindo as variáveis a partir das 8 primeiras linhas
                        metadata = {
                            "REGIAO": first_8_lines[0].split(';')[1].strip(),
                            "UF": first_8_lines[1].split(';')[1].strip(),
                            "ESTACAO": first_8_lines[2].split(';')[1].strip(),
                            "CODIGO_WMO": first_8_lines[3].split(';')[1].strip(),
                            "LATITUDE": first_8_lines[4].split(';')[1].strip(),
                            "LONGITUDE": first_8_lines[5].split(';')[1].strip(),
                            "ALTITUDE": first_8_lines[6].split(';')[1].strip(),
                            "DATA_FUNDACAO": first_8_lines[7].split(';')[1].strip()
                        }

                        # Lê o resto do arquivo ignorando as 8 primeiras linhas
                        temp_df = pd.read_csv(
                            file_path,
                            skiprows=8,
                            encoding='latin1',
                            sep=';',  # Especifica o delimitador correto
                            engine='python'         # Usa o motor Python para maior flexibilidade
                        )
                        # Adiciona as variáveis extraídas como novas colunas
                        for key, value in metadata.items():
                            temp_df[key] = value

                        # Adiciona o arquivo lido ao DataFrame concatenado
                        concatenated_df = pd.concat([concatenated_df, temp_df], ignore_index=True)
                    except Exception as e:
                        print(f"Erro ao processar {file_path}: {e}")
                else:
                    print(f"Arquivo já processado ou não encontrado: {file_path}")
        else:
            print(f"Ano não encontrado no arquivo {base_path}")

    # Remove duplicatas do DataFrame final considerando todas as colunas
    concatenated_df = concatenated_df.drop_duplicates()

    return concatenated_df
        
# retira warnings
import warnings
warnings.filterwarnings('ignore')

# Carrega os dados de acidentes com estações climáticas
datas_climaticas = load_and_concat_unique_data_with_metadata(acidentes_com_estacoes)
datas_climaticas.head()

In [ ]:
# # verifica duplicatas
# print(datas_climaticas.duplicated().sum())
# # verifica Data value counts
# print(datas_climaticas['Data'].value_counts())
datas_climaticas

In [ ]:
datas_climaticas.to_csv('./secundario/datas_climaticas.csv', index=False) 

In [ ]:
import pandas as pd
import numpy as np # type: ignore
from geopy.distance import geodesic # type: ignore
from datetime import datetime, timedelta

def find_nearest_weather_station_with_time(accidents_df, weather_stations_df, radius_km=50):
    # Função para converter vírgulas para ponto e depois para float
    def convert_to_float(value):
        if isinstance(value, str):  # Verifica se o valor é uma string
            value = value.replace(',', '.')  # Substitui vírgula por ponto
        return float(value) if value else 0.0  # Converte para float, ou 0.0 se for None ou vazio

    # Copiar os DataFrames de entrada para evitar modificar os originais
    accidents_df = accidents_df.copy()
    weather_stations_df = weather_stations_df.copy()

    # Remover linhas com valores NaN em coordenadas
    accidents_df = accidents_df.dropna(subset=['latitude', 'longitude'])
    weather_stations_df = weather_stations_df.dropna(subset=['LATITUDE', 'LONGITUDE'])

    # Converter as colunas de latitude e longitude para float (com correção de vírgula)
    weather_stations_df['LATITUDE'] = weather_stations_df['LATITUDE'].apply(convert_to_float)
    weather_stations_df['LONGITUDE'] = weather_stations_df['LONGITUDE'].apply(convert_to_float)

    # Converter as colunas de data e hora para datetime
    accidents_df['data_hora'] = pd.to_datetime(accidents_df['data_hora'])
    
    # Ajuste da coluna de hora no dataframe de clima
    weather_stations_df['datetime'] = pd.to_datetime(weather_stations_df['Data'] + ' ' + weather_stations_df['Hora UTC'].str.replace(' UTC', ''), format='%Y/%m/%d %H%M')

    # Lista para armazenar resultados
    merged_data = []

    # Itera sobre cada linha do DataFrame de acidentes
    for _, accident_row in accidents_df.iterrows():
        accident_location = (accident_row['latitude'], accident_row['longitude'])
        accident_time = accident_row['data_hora']

        # print(f"Procurando estação climática mais próxima para o acidente em {accident_location} na hora {accident_time}")

        # Calcula a distância para cada estação climática
        weather_stations_df['distance'] = weather_stations_df.apply(
            lambda row: geodesic(accident_location, (row['LATITUDE'], row['LONGITUDE'])).kilometers, axis=1
        )

        # Filtra estações dentro do raio especificado
        nearby_stations = weather_stations_df[weather_stations_df['distance'] <= radius_km]

        if not nearby_stations.empty:
            # Encontrar a estação mais próxima em termos de data/hora
            nearby_stations['time_diff'] = (nearby_stations['datetime'] - accident_time).abs()

            # Encontra a estação com a menor diferença de tempo
            nearest_station = nearby_stations.loc[nearby_stations['time_diff'].idxmin()]
            
            # Combina os dados do acidente e da estação climática
            combined_row = {**accident_row.to_dict(), **nearest_station.to_dict()}
        else:
            # Se nenhuma estação estiver dentro do raio, preenche com NaN
            combined_row = {**accident_row.to_dict(), **{col: np.nan for col in weather_stations_df.columns}}

        merged_data.append(combined_row)

    # Retorna o DataFrame combinado
    return pd.DataFrame(merged_data)


acidentes_com_estacoes = find_nearest_weather_station_with_time(acidentes_df, datas_climaticas)
acidentes_com_estacoes


In [ ]:
acidentes_com_estacoes.to_csv('./secundario/acidentes_com_estacoes.csv', index=False)

In [ ]:
# Tente com 'latin1' ou 'iso-8859-1'
# clima_df = pd.read_csv('../data/INMET/2023/INMET_SE_SP_A768_TUPA_01-01-2023_A_31-12-2023.csv', 
#                        sep=';', 
#                        skiprows=8, 
#                        encoding='latin1')  # Ou experimente 'iso-8859-1'
# clima_df.head(2)

In [ ]:
# acidentes_df.columns

In [ ]:
# df = pd.read_csv('rodo_estacoes.csv')
# df